# Capstone Project- Battle of Neighborhoods #
## Business Objective:  ##
### Finding a suitable neighborhoods for an Indian restaurant in Singapore ###

This notebook is structured into 6 sections
* Section 1 Importing libraries  
* Section 2 Data wrangling     
* Section 3  Getting the geospatial data. 
* Section 4 Analysing the neighborhoods
* Section 5 Clustering 
* Section 6 Examining the clusters

**Section 1: Importing necessary libraries**   
*sklearn & geopy was installed separately to the kernel*

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Import Completed.')

Import Completed.


**Section 2: Data import**  
*The singapore MRT location data was sourced from a kaggle notebook by LY Xuan*        
*(https://www.kaggle.com/yxlee245/singapore-train-station-coordinates)*

In [10]:
df=pd.read_csv('MRT_Data.csv')
df.head()

,Neighborhood,Type,Latitude,Longitude
0,Jurong East,MRT,1.333207,103.742308
1,Bukit Batok,MRT,1.349069,103.749596
2,Bukit Gombak,MRT,1.359043,103.751863
3,Choa Chu Kang,MRT,1.385417,103.744316
4,Yew Tee,MRT,1.397383,103.747523


**Section 3: Getting the geospatial data**. 
  
**3a. Identifying the coordinates of Singapore**

In [4]:
address = 'Singapore, SG'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Singapore are 1.357107, 103.8194992.


**3b. Connecting to Foursquare**

In [11]:
CLIENT_ID = 'B2UZKDQWMH0KAJSAP2BP15R31KQTEYIB4AJLTTTGNH03HFH1' # my Foursquare ID
CLIENT_SECRET = '0IIY5LNJAHYU33BB1HA5LZ1YJPQJUEJQATRMV54CGGKVX5XB' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B2UZKDQWMH0KAJSAP2BP15R31KQTEYIB4AJLTTTGNH03HFH1
CLIENT_SECRET:0IIY5LNJAHYU33BB1HA5LZ1YJPQJUEJQATRMV54CGGKVX5XB


**Section 4: Analysing the neighbourhoods**. 
  
**4a. Exploring the neighbourhoods**

In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

singapore_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Jurong East
Bukit Batok
Bukit Gombak
Choa Chu Kang
Yew Tee
Kranji
Marsiling
Woodlands
Admiralty
Sembawang
Yishun
Khatib
Yio Chu Kang
Ang Mo Kio
Bishan
Braddell
Toa Payoh
Novena
Newton
Orchard
Somerset
Dhoby Ghaut
City Hall
Raffles Place
Marina Bay
Marina South Pier
Tuas Link
Tuas West Road
Tuas Crescent
Gul Circle
Joo Koon
Pioneer
Boon Lay
Lakeside
Chinese Garden
Clementi
Dover
Buona Vista
Commonwealth
Queenstown
Redhill
Tiong Bahru
Outram Park
Tanjong Pagar
Bugis
Lavender
Kallang
Aljunied
Paya Lebar
Eunos
Kembangan
Bedok
Tanah Merah
Simei
Tampines
Pasir Ris
Expo
Changi Airport
HarbourFront
Chinatown
Clarke Quay
Little India
Farrer Park
Boon Keng
Potong Pasir
Woodleigh
Serangoon
Kovan
Hougang
Buangkok
Sengkang
Punggol
Bras Basah
Esplanade
Promenade
Nicoll Highway
Stadium
Mountbatten
Dakota
MacPherson
Tai Seng
Bartley
Lorong Chuan
Marymount
Caldecott
Botanic Gardens
Farrer Road
Holland Village
one-north
Kent Ridge
Haw Par Villa
Pasir Panjang
Labrador Park
Telok Blangah
Bayfront
Bukit Pa

In [13]:
singapore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jurong East,1.333207,103.742308,UNIQLO,1.333175,103.743160,Clothing Store
1,Jurong East,1.333207,103.742308,Johan Paris,1.334083,103.742384,Bakery
2,Jurong East,1.333207,103.742308,Tonkatsu by Ma Maison とんかつ マメゾン (Tonkatsu by M...,1.333668,103.742818,Japanese Restaurant
3,Jurong East,1.333207,103.742308,MUJI 無印良品,1.333187,103.743064,Furniture / Home Store
4,Jurong East,1.333207,103.742308,Books Kinokuniya,1.333482,103.743163,Bookstore


**4b. Top venues for each neighbourhoods**

In [14]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")
singapore_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
singapore_onehot.insert(loc=0, column='Neighborhood', value=singapore_venues['Neighborhood'] )
singapore_onehot.shape

singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Lounge,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Yunnan Restaurant
0,Admiralty,0.0,0.0,0.0,0.018868,0.018868,0.0,0.0,0.0,0.037736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aljunied,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.108696,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ang Mo Kio,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.025641,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bakau,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bangkit,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**4C. Neighbourhoods with atleast one Indian Restaurant**

In [15]:
col=['Neighborhood']
for column in singapore_onehot.columns:
    if column.__contains__("Indian Restaurant"):
        col.append(column)

singapore_restaurants=singapore_onehot[col]
sg_ind_rest1=singapore_restaurants.groupby('Neighborhood').sum().reset_index()
#sg_ind_rest=singapore_restaurants.groupby('Neighborhood').count().reset_index()
sg_ind_rest=sg_ind_rest1[['Neighborhood','Indian Restaurant']]

sg_ind_rest.sort_values(by=['Indian Restaurant'], ascending=False)

sg_ind_rest1=sg_ind_rest.loc[sg_ind_rest['Indian Restaurant'] > 1]
sg_ind_rest1

,Neighborhood,Indian Restaurant
0,Admiralty,2
7,Beauty World,2
15,Boon Lay,2
20,Bugis,2
24,Buona Vista,4
35,Commonwealth,2
46,Expo,2
49,Farrer Park,7
60,Jalan Besar,20
63,Jurong East,2


**4d. Top venues in each neighbourhood**

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiralty,Japanese Restaurant,Café,Shopping Mall,Frozen Yogurt Shop,Seafood Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Electronics Store,Indian Restaurant
1,Aljunied,Asian Restaurant,Chinese Restaurant,Coffee Shop,Dim Sum Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Food Court,Seafood Restaurant,Other Nightlife,Farmers Market
2,Ang Mo Kio,Coffee Shop,Food Court,Dessert Shop,Bubble Tea Shop,Japanese Restaurant,Supermarket,Convenience Store,Pharmacy,Sushi Restaurant,Noodle House
3,Bakau,Breakfast Spot,Shopping Mall,Malay Restaurant,Fast Food Restaurant,Sandwich Place,Indonesian Restaurant,Bus Station,Pharmacy,Japanese Restaurant,Thai Restaurant
4,Bangkit,Food Court,Lottery Retailer,Miscellaneous Shop,Market,Bakery,Bike Trail,Fast Food Restaurant,Coffee Shop,Noodle House,Park


**Section 5: Clustering neighbourhoods in Singapore**   
  
**5a. Running k means clustering**

In [18]:
# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 4, 1, 4, 1, 1, 1, 1, 4], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))

singapore_merged = df

# merge the dataframes to add latitude/longitude for each neighborhood
singapore_merged = singapore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')
singapore_merged.head()

,Neighborhood,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jurong East,MRT,1.333207,103.742308,1,Japanese Restaurant,Café,Coffee Shop,Chinese Restaurant,Food Court,Shopping Mall,Clothing Store,Korean Restaurant,Multiplex,Department Store
1,Bukit Batok,MRT,1.349069,103.749596,4,Coffee Shop,Food Court,Chinese Restaurant,Indian Restaurant,Sandwich Place,Bowling Alley,Café,Skate Park,Shopping Mall,Fast Food Restaurant
2,Bukit Gombak,MRT,1.359043,103.751863,4,Food Court,Convenience Store,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,Stadium,Sandwich Place,Fast Food Restaurant,Lake,Supermarket
3,Choa Chu Kang,MRT,1.385417,103.744316,1,Italian Restaurant,Smoke Shop,Fast Food Restaurant,Coffee Shop,Accessories Store,Sandwich Place,Lounge,Noodle House,Sushi Restaurant,Supermarket
4,Yew Tee,MRT,1.397383,103.747523,1,Fast Food Restaurant,Japanese Restaurant,Shopping Mall,Café,Sandwich Place,Diner,Food Court,Cosmetics Shop,Pool,Stationery Store


**5b. Mapping the clusters**

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Neighborhood'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Section 6: Analysing the clusters**.   
  
**Cluster 1: Residential Areas**

In [79]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 0, singapore_merged.columns[[0] + list(range(5, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Lakeside,Convenience Store,Snack Place,Basketball Court,Bus Station,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop
82,Lorong Chuan,Indonesian Restaurant,Basketball Court,Bus Station,Café,Yunnan Restaurant,Food,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market
124,Jelapang,Bus Station,Breakfast Spot,Shopping Mall,Food & Drink Shop,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food
130,Compassvale,Park,Garden,Sculpture Garden,Playground,Coffee Shop,Bus Station,Flower Shop,Fast Food Restaurant,Field,Filipino Restaurant
133,Kangkar,Bus Station,Breakfast Spot,Supermarket,Park,Gym / Fitness Center,Gym Pool,Fast Food Restaurant,Shopping Mall,Thai Restaurant,Malay Restaurant
141,Tongkang,Bus Station,Grocery Store,Basketball Court,Seafood Restaurant,Playground,Bookstore,Flower Shop,Field,Filipino Restaurant,Fish & Chips Shop
142,Renjong,Playground,Grocery Store,Light Rail Station,Bus Station,Food,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market


**Cluster 2: Small Towns**

In [80]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 1, singapore_merged.columns[[0] + list(range(5, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bukit Batok,Coffee Shop,Food Court,Chinese Restaurant,Indian Restaurant,Sandwich Place,Bowling Alley,Café,Skate Park,Shopping Mall,Fast Food Restaurant
2,Bukit Gombak,Food Court,Convenience Store,Vegetarian / Vegan Restaurant,Chinese Restaurant,Coffee Shop,Stadium,Sandwich Place,Fast Food Restaurant,Lake,Supermarket
6,Marsiling,Food Court,Coffee Shop,Grocery Store,Cosmetics Shop,Music Venue,Flea Market,Paintball Field,Fast Food Restaurant,Hainan Restaurant,Bakery
9,Sembawang,Fast Food Restaurant,Coffee Shop,Convenience Store,Shopping Mall,Chinese Restaurant,Basketball Court,Bistro,Supermarket,Sporting Goods Shop,Japanese Restaurant
11,Khatib,Coffee Shop,Gym / Fitness Center,Supermarket,Asian Restaurant,Rest Area,Grocery Store,Shopping Mall,Park,Café,Bakery
12,Yio Chu Kang,Pool,Food Court,Chinese Restaurant,Noodle House,Fast Food Restaurant,Gym / Fitness Center,Bakery,Food,Coffee Shop,College Auditorium
13,Ang Mo Kio,Coffee Shop,Food Court,Dessert Shop,Bubble Tea Shop,Japanese Restaurant,Supermarket,Convenience Store,Pharmacy,Sushi Restaurant,Noodle House
14,Bishan,Food Court,Coffee Shop,Bubble Tea Shop,Café,Japanese Restaurant,Chinese Restaurant,Asian Restaurant,Supermarket,Ice Cream Shop,Cosmetics Shop
16,Toa Payoh,Coffee Shop,Food Court,Pool,Dessert Shop,Snack Place,Chinese Restaurant,Kids Store,Frozen Yogurt Shop,Bubble Tea Shop,Bus Station
29,Gul Circle,Coffee Shop,Electronics Store,Train Station,Park,Yunnan Restaurant,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


**Cluster 3: Medium sized towns**

In [81]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 2, singapore_merged.columns[[0] + list(range(5, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jurong East,Japanese Restaurant,Café,Coffee Shop,Chinese Restaurant,Food Court,Shopping Mall,Clothing Store,Korean Restaurant,Multiplex,Department Store
3,Choa Chu Kang,Italian Restaurant,Smoke Shop,Fast Food Restaurant,Coffee Shop,Accessories Store,Sandwich Place,Lounge,Noodle House,Sushi Restaurant,Supermarket
4,Yew Tee,Fast Food Restaurant,Japanese Restaurant,Shopping Mall,Café,Sandwich Place,Diner,Food Court,Cosmetics Shop,Pool,Stationery Store
5,Kranji,Racetrack,Bus Line,Go Kart Track,Bus Station,Bus Stop,Food Truck,Food Stand,Food Court,French Restaurant,Fried Chicken Joint
7,Woodlands,Japanese Restaurant,Café,Shopping Mall,Frozen Yogurt Shop,Seafood Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Electronics Store,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
151,Teck Lee,Light Rail Station,Park,Bridge,Pool,Yoga Studio,Fast Food Restaurant,Sports Bar,Spa,Bowling Alley,Steakhouse
152,Punggol Point,Light Rail Station,Campground,Yunnan Restaurant,Farmers Market,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop
153,Samudera,Soccer Field,Bar,Sporting Goods Shop,Club House,Beer Garden,Beer Bar,Chinese Restaurant,Light Rail Station,Harbor / Marina,Farm
154,Nibong,Soccer Field,Bar,Beer Bar,Coffee Shop,Scenic Lookout,Farm,Chinese Restaurant,Beer Garden,Light Rail Station,Harbor / Marina


**Cluster 4: Emerging new town**

In [82]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 3, singapore_merged.columns[[0] + list(range(5, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
144,Meridian,Chinese Restaurant,Supermarket,Yunnan Restaurant,Food,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop


**Cluster 5: Downtown & nearby suburbs**

In [83]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 4, singapore_merged.columns[[0] + list(range(5, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Yishun,Chinese Restaurant,Food Court,Supermarket,Coffee Shop,Fried Chicken Joint,Japanese Restaurant,Hainan Restaurant,Italian Restaurant,Indian Restaurant,Bookstore
15,Braddell,Noodle House,Chinese Restaurant,Food Court,Café,Seafood Restaurant,Thai Restaurant,Fast Food Restaurant,Asian Restaurant,Hakka Restaurant,Bakery
18,Newton,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Indian Restaurant,Noodle House,Dance Studio,Pool,Pizza Place,Nightclub
26,Tuas Link,Sporting Goods Shop,Sports Club,Chinese Restaurant,Harbor / Marina,Gastropub,Train Station,Bus Station,Flower Shop,Field,Filipino Restaurant
34,Chinese Garden,Gym Pool,Coffee Shop,Indian Restaurant,Asian Restaurant,Food Court,Gift Shop,Café,Lottery Retailer,Train Station,Chinese Restaurant
37,Buona Vista,Chinese Restaurant,Food Court,Japanese Restaurant,Indian Restaurant,Coffee Shop,Bakery,Salad Place,BBQ Joint,Shopping Mall,Café
38,Commonwealth,Asian Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Indian Restaurant,Coffee Shop,Diner,Paper / Office Supplies Store,Fast Food Restaurant,Fish & Chips Shop
39,Queenstown,Food Court,Chinese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Coffee Shop,Spa,Café,Train Station,Seafood Restaurant,Italian Restaurant
40,Redhill,Chinese Restaurant,Coffee Shop,Asian Restaurant,Malay Restaurant,Gym,Hainan Restaurant,Bistro,Train Station,Dive Shop,Park
41,Tiong Bahru,Chinese Restaurant,Bakery,Food Court,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Supermarket,Café,Halal Restaurant,Fast Food Restaurant


**Clusters with Indian Restaurants**

In [106]:
singapore_clusters=singapore_merged[['Neighborhood','Cluster Labels']]

ind_res_cluster = pd.merge(sg_ind_rest1,singapore_clusters,how='inner',on='Neighborhood')
ind_res_cluster

,Neighborhood,Indian Restaurant,Cluster Labels
0,Admiralty,2,2
1,Beauty World,2,4
2,Boon Lay,2,2
3,Bugis,2,2
4,Buona Vista,4,4
5,Commonwealth,2,4
6,Expo,2,2
7,Farrer Park,7,4
8,Jalan Besar,20,4
9,Jurong East,2,2
